In [1]:
!pip install catboost[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 16.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/download')

Mounted at /content/download


In [3]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
from scipy.special import logsumexp

from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import mean_squared_error, classification_report

In [4]:
df = pd.read_csv("/content/download/MyDrive/1-09-1-20.csv")
print(df.shape)
df.tail()

(3730870, 43)


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,32,33,34,35,36,37,38,39,40,41
3730865,3730865,1674238247671,2023-01-20 18:10:47,21337.6,0.167,21337.5,0.100,21337.4,0.014,21336.9,...,21338.2,0.022,21338.3,0.028,21338.4,0.235,21338.5,1.657,21338.6,0.234
3730866,3730866,1674238247921,2023-01-20 18:10:47,21337.5,0.108,21337.4,0.014,21336.9,0.257,21336.8,...,21338.2,0.022,21338.3,0.036,21338.4,0.236,21338.5,1.657,21338.6,0.235
3730867,3730867,1674238248172,2023-01-20 18:10:48,21337.4,0.891,21337.2,0.552,21337.1,1.095,21336.9,...,21338.1,0.002,21338.2,0.048,21338.3,0.036,21338.4,0.001,21338.5,1.658
3730868,3730868,1674238248422,2023-01-20 18:10:48,21337.4,2.456,21337.3,1.149,21337.2,0.002,21336.9,...,21338.1,0.002,21338.2,0.048,21338.3,0.036,21338.4,0.001,21338.5,1.658
3730869,3730869,1674238248672,2023-01-20 18:10:48,21337.4,2.513,21337.3,1.149,21337.2,0.002,21336.9,...,21338.1,0.002,21338.2,0.048,21338.3,0.036,21338.4,0.001,21338.5,1.047


In [5]:
new_columns = [
    'index', 'timestamp', 'time', 'bid1', 'bid1vol', 'bid2', 'bid2vol',
    'bid3', 'bid3vol', 'bid4', 'bid4vol', 'bid5', 'bid5vol',
    'bid6', 'bid6vol', 'bid7', 'bid7vol', 'bid8', 'bid8vol',
    'bid9', 'bid9vol', 'bid10', 'bid10vol', 'ask1', 'ask1vol',
    'ask2', 'ask2vol', 'ask3', 'ask3vol', 'ask4', 'ask4vol',
    'ask5', 'ask5vol', 'ask6', 'ask6vol', 'ask7', 'ask7vol',
    'ask8', 'ask8vol', 'ask9', 'ask9vol', 'ask10', 'ask10vol'
]
df.columns = new_columns
df['mid'] = 0.5*(df['bid1'] + df['ask1'])

print(df.head())

   index      timestamp                 time     bid1  bid1vol     bid2  \
0      0  1673302660926  2023-01-09 22:17:40  17181.6   23.371  17181.5   
1      1  1673302661177  2023-01-09 22:17:41  17181.6   24.232  17181.5   
2      2  1673302661427  2023-01-09 22:17:41  17181.6   24.403  17181.5   
3      3  1673302661678  2023-01-09 22:17:41  17181.6   24.874  17181.5   
4      4  1673302661928  2023-01-09 22:17:41  17181.6   24.403  17181.5   

   bid2vol     bid3  bid3vol     bid4  ...  ask6vol     ask7  ask7vol  \
0    0.746  17181.4    5.428  17181.2  ...    5.168  17182.3     0.02   
1    0.694  17181.4    5.428  17181.2  ...    6.043  17182.3     0.02   
2    0.694  17181.4    5.428  17181.2  ...    6.043  17182.3     0.02   
3    0.694  17181.4    5.428  17181.2  ...    6.043  17182.3     0.02   
4    0.694  17181.4    5.428  17181.2  ...    6.043  17182.3     0.02   

      ask8  ask8vol     ask9  ask9vol    ask10  ask10vol       mid  
0  17182.4    6.692  17182.5    1.904  17

In [6]:
interval_set = 5 * 1000 #predicted timeslot
df['timestamp_predict'] = df['timestamp'] + interval_set
df['mom'] = df['mid']
# Use numpy to find the closest indices efficiently
timestamps = df['timestamp'].values
predicted_timestamps = df['timestamp_predict'].values
closest_indices = np.searchsorted(timestamps, predicted_timestamps)

for ind, row in df.iterrows():
    if ind == len(df) - 1:
        df.at[ind, 'mom'] = 0
        break

    # Find the closest timestamp index
    closest_index = closest_indices[ind]
    if closest_index >= len(df):
        closest_index = len(df) - 1

    if row['mid'] < df.loc[closest_index, 'mid']:
        df.at[ind, 'mom'] = 1
    if row['mid'] > df.loc[closest_index, 'mid']:
        df.at[ind, 'mom'] = -1
    if row['mid'] == df.loc[closest_index, 'mid']:
        df.at[ind, 'mom'] = 0


In [7]:
## y is binary.
display(df['mom'].describe())

count    3.730870e+06
mean    -4.076261e-03
std      8.370129e-01
min     -1.000000e+00
25%     -1.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: mom, dtype: float64

In [8]:
df.to_csv("/content/download/MyDrive/download/train_hft.csv")

In [25]:
df = pd.read_csv("/content/download/MyDrive/download/train_hft.csv")

In [26]:
bid_cols = ['bid1', 'bid2', 'bid3', 'bid4', 'bid5', 'bid6', 'bid7', 'bid8', 'bid9', 'bid10']
bid_vol_cols = ['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol', 'bid6vol', 'bid7vol', 'bid8vol', 'bid9vol', 'bid10vol']
ask_cols = ['ask1', 'ask2', 'ask3', 'ask4', 'ask5', 'ask6', 'ask7', 'ask8', 'ask9', 'ask10']
ask_vol_cols = ['ask1vol','ask2vol', 'ask3vol', 'ask4vol', 'ask5vol', 'ask6vol', 'ask7vol', 'ask8vol', 'ask9vol', 'ask10vol']

group_cols = {"bid_cols":bid_cols,"bid_vol_cols":bid_vol_cols,"ask_cols":ask_cols,"ask_vol_cols":ask_vol_cols}

*  Additional features could include: rank, which bid number is the max/min, etc'
* features between the aggregated features (e.g. max bid div max ask..)

In [27]:
level = 10

In [28]:
def feature_engineering(ds, level, num_lags):
    out_ds = pd.DataFrame()

    for i in range(1, level + 1):
        out_ds['log_return_ask_{0}'.format(i)] = np.log(ds['ask{0}'.format(i)].pct_change() + 1)
        out_ds['log_return_bid_{0}'.format(i)] = np.log(ds['bid{0}'.format(i)].pct_change() + 1)

        out_ds['log_ask_{0}_div_bid_{0}'.format(i)] = np.log(ds['ask{0}'.format(i)] / ds['bid{0}'.format(i)])
        out_ds['log_ask_{0}_vol_div_bid_{0}'.format(i)] = np.log(ds['ask{0}vol'.format(i)] / ds['bid{0}vol'.format(i)])

        out_ds['log_ask_{0}_vol'.format(i)] = np.log(ds['ask{0}vol'.format(i)])
        out_ds['log_bid_{0}_vol'.format(i)] = np.log(ds['bid{0}vol'.format(i)])

        if i != 1:
            out_ds['log_ask_{0}_div_ask_1'.format(i)] = np.log(ds['ask{0}'.format(i)] / ds['ask1'])
            out_ds['log_bid_{0}_div_bid_1'.format(i)] = np.log(ds['bid{0}'.format(i)] / ds['bid1'])
            out_ds['log_ask_{0}_vol_div_ask_1'.format(i)] = np.log(ds['ask{0}vol'.format(i)] / ds['ask1vol'])
            out_ds['log_bid_{0}_vol_div_bid_1'.format(i)] = np.log(ds['bid{0}vol'.format(i)] / ds['bid1vol'])

    out_ds['log_total_volume_ask'] = np.log(ds[['ask{0}vol'.format(x) for x in list(range(1, level + 1))]].sum(axis = 1))
    out_ds['log_total_volume_bid'] = np.log(ds[['bid{0}vol'.format(x) for x in list(range(1, level + 1))]].sum(axis = 1))

    mid_price = (ds['ask1'] + ds['bid1']) / 2
    out_ds['log_return_mid_price'] = np.log(mid_price.pct_change() + 1).shift(-1)
    out_ds['mom'] = ds['mom']
    cols_features = out_ds.columns.drop('mom')
    out_ds = out_ds.assign(**{
        '{}_(t-{})'.format(col, t): out_ds[col].shift(t)
        for t in list(range(1, num_lags))
        for col in cols_features})

    return out_ds.dropna()

# Split back into train and test, and build model

In [29]:
end_time = df['timestamp'].tolist()[-1] - interval_set
df = df[df['timestamp'] <= end_time]
df = feature_engineering(df, 10, 3)

<ipython-input-28-f5bbca8c0770>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out_ds = out_ds.assign(**{
<ipython-input-28-f5bbca8c0770>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out_ds = out_ds.assign(**{
<ipython-input-28-f5bbca8c0770>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  o

In [30]:
split_index = int(len(df) * 0.8)
train = df.iloc[:split_index]
test = df.iloc[split_index:]

In [31]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [32]:
X = train.drop(['mom'],axis=1)
y = train['mom']

In [33]:
from sklearn.model_selection import KFold
model = CatBoostClassifier(iterations=1000, depth=5, learning_rate=0.1, loss_function='MultiClass', verbose=False, task_type="GPU")

# Define k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform k-fold cross-validation
cv_results = []
models = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)
    models.append(model)
    # Evaluate the model
    score = model.score(X_test, y_test)
    cv_results.append(score)


In [34]:
print(cv_results)

[0.6023526810244314, 0.6031795756656922, 0.603620159648873, 0.6032532855336008, 0.6036452880129327]


In [35]:
from scipy.stats import mode

y_test = test['mom'].tolist()
X_test = test.drop(['mom'],axis=1)
predictions = np.zeros((X_test.shape[0], len(models)))

for i, model in enumerate(models):
    predictions[:, i] = model.predict(X_test).flatten()

final_predictions, _ = mode(predictions, axis=1)

# Flatten the final predictions array
final_predictions = final_predictions.flatten()

# Convert final_predictions to a list if needed
final_predictions_list = final_predictions.tolist()

In [36]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

accuracy = accuracy_score(y_test, final_predictions)
print("Accuracy:", accuracy)

# Calculate F1-score
f1 = f1_score(y_test, final_predictions, average='weighted')
print("F1-Score:", f1)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.5619952557728132
F1-Score: 0.5618851912317464
Confusion Matrix:
 [[162805  51022  58259]
 [ 56097 110046  47718]
 [ 61252  52478 146493]]


## Features importances
